In [1]:
import pandas as pd
import re

In [2]:
movies = pd.read_csv("movies.csv",header=0)
ratings = pd.read_csv("ratings.csv",header = 0)
tags = pd.read_csv("tags.csv",header = 0)
links = pd.read_csv("links.csv",header = 0)


In [3]:
tags_agregadas = tags.groupby('movieId')['tag'].apply(lambda x: ' | '.join(x.dropna().astype(str))).reset_index()

info_filmes = pd.merge(movies, links, on='movieId', how='left')

df_final = pd.merge(info_filmes, ratings, on='movieId', how='inner')

df_final['rating'] = pd.to_numeric(df_final['rating'])
df_final['timestamp'] = pd.to_datetime(df_final['timestamp'], unit='s')


In [ ]:
df_final

,movieId,title,genres,imdbId,tmdbId,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,10,2.5,2007-01-20 03:53:51
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,11,3.0,1996-12-08 22:44:36
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,17,4.0,2002-07-22 02:42:31
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,19,3.0,2000-11-20 07:14:48
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,20,5.0,2019-03-21 16:03:50
...,...,...,...,...,...,...,...,...
32000199,292731,The Monroy Affaire (2022),Drama,26812510,1032473.0,50685,4.0,2023-10-12 01:39:39
32000200,292737,Shelter in Solitude (2023),Comedy|Drama,14907358,986674.0,165364,1.5,2023-10-12 21:25:47
32000201,292753,Orca (2023),Drama,12388280,948139.0,150812,4.0,2023-10-12 19:55:30
32000202,292755,The Angry Breed (1968),Drama,64027,182776.0,91097,1.0,2023-10-12 21:08:48


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000204 entries, 0 to 32000203
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   movieId    int64         
 1   title      object        
 2   genres     object        
 3   imdbId     int64         
 4   tmdbId     float64       
 5   userId     int64         
 6   rating     float64       
 7   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 1.9+ GB


In [4]:
contagem_f = df_final['movieId'].value_counts()
manter_f = contagem_f[contagem_f >= 5].index
df_final_filtrado = df_final[df_final['movieId'].isin(manter_f)]
df_final_limitado = df_final_filtrado.groupby('movieId').head(5)

In [ ]:
df_final_limitado.info()

In [ ]:
df_final_limitado

,movieId,title,genres,imdbId,tmdbId,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,10,2.5,2007-01-20 03:53:51
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,11,3.0,1996-12-08 22:44:36
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,17,4.0,2002-07-22 02:42:31
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,19,3.0,2000-11-20 07:14:48
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,20,5.0,2019-03-21 16:03:50
...,...,...,...,...,...,...,...,...
32000123,292349,Totally Killer (2023),Comedy|Horror,11426232,974931.0,5029,3.0,2023-10-12 08:19:09
32000124,292349,Totally Killer (2023),Comedy|Horror,11426232,974931.0,7858,1.0,2023-10-12 00:29:11
32000125,292349,Totally Killer (2023),Comedy|Horror,11426232,974931.0,11190,4.0,2023-10-11 21:13:33
32000126,292349,Totally Killer (2023),Comedy|Horror,11426232,974931.0,36418,3.0,2023-10-11 21:09:43


In [5]:
def extract_year(title):
    ano_titulo = re.search(r'\((\d{4})\)', title)
    if ano_titulo:
        return int(ano_titulo.group(1))
    return None

df_final_limitado['release_year'] = df_final_limitado['title'].apply(extract_year)
df_final_filtrado_ano = df_final_limitado[df_final_limitado['release_year'] >= 2015]

/tmp/ipython-input-400725431.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_limitado['release_year'] = df_final_limitado['title'].apply(extract_year)


In [6]:
df_final_filtrado_ano

,movieId,title,genres,imdbId,tmdbId,userId,rating,timestamp,release_year
264558,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,28,3.0,2020-01-04 03:15:09,2015.0
264559,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,75,5.0,2023-08-27 05:53:59,2015.0
264560,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,76,4.0,2019-02-08 20:50:53,2015.0
264561,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,151,4.0,2015-10-28 11:02:11,2015.0
264562,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,198,3.5,2015-12-30 22:49:01,2015.0
...,...,...,...,...,...,...,...,...,...
288930,288513,Barbie (2023),Comedy,1517268,346698.0,28,5.0,2023-09-01 01:57:22,2023.0
288931,288513,Barbie (2023),Comedy,1517268,346698.0,79,4.5,2023-08-25 15:24:09,2023.0
288932,288513,Barbie (2023),Comedy,1517268,346698.0,326,4.0,2023-07-24 04:47:49,2023.0
288933,288513,Barbie (2023),Comedy,1517268,346698.0,458,4.0,2023-09-27 20:50:22,2023.0


In [8]:
df_final_generos = df_final_filtrado_ano[df_final_filtrado_ano['genres'] != '(no genres listed)']

In [10]:
df_final_generos

,movieId,title,genres,imdbId,tmdbId,userId,rating,timestamp,release_year
264558,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,28,3.0,2020-01-04 03:15:09,2015.0
264559,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,75,5.0,2023-08-27 05:53:59,2015.0
264560,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,76,4.0,2019-02-08 20:50:53,2015.0
264561,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,151,4.0,2015-10-28 11:02:11,2015.0
264562,111781,Mission: Impossible - Rogue Nation (2015),Action|Adventure|Thriller,2381249,177677.0,198,3.5,2015-12-30 22:49:01,2015.0
...,...,...,...,...,...,...,...,...,...
288930,288513,Barbie (2023),Comedy,1517268,346698.0,28,5.0,2023-09-01 01:57:22,2023.0
288931,288513,Barbie (2023),Comedy,1517268,346698.0,79,4.5,2023-08-25 15:24:09,2023.0
288932,288513,Barbie (2023),Comedy,1517268,346698.0,326,4.0,2023-07-24 04:47:49,2023.0
288933,288513,Barbie (2023),Comedy,1517268,346698.0,458,4.0,2023-09-27 20:50:22,2023.0


In [9]:
df_final_generos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3315 entries, 264558 to 288934
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   movieId       3315 non-null   int64         
 1   title         3315 non-null   object        
 2   genres        3315 non-null   object        
 3   imdbId        3315 non-null   int64         
 4   tmdbId        3315 non-null   float64       
 5   userId        3315 non-null   int64         
 6   rating        3315 non-null   float64       
 7   timestamp     3315 non-null   datetime64[ns]
 8   release_year  3315 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 259.0+ KB
